In [1]:
# Modify the file 00_setup.py to define input/output file paths on your system
# The information in 00_setup.py will be used across notebooks
from importlib.machinery import SourceFileLoader
setup = SourceFileLoader("setup", "./00_setup.py").load_module()

# XGBoost, Modified DGI Hierarchical - Pairs
Do an alternative hierarchical blending, NAICS + one other level.  Do all pairs

*This script takes about 5 minutes on my MacBook Air*

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import warnings
import matplotlib.pyplot as plt
import pickle

In [3]:
from pathlib import Path

In [4]:
import sklearn as sk
from sklearn.model_selection import RandomizedSearchCV
from sklearn.inspection import permutation_importance, partial_dependence
from scikitplot.metrics import plot_lift_curve

In [5]:
from sba_gnn.sba_gnn import sg_plot, sg_blender
from sba_gnn.sba_gnn.sg_blender import HierarchicalEncoder # Target encoder

## Import Processed Datasets

In [6]:
sba_loans = pd.read_parquet(Path(setup.temp_path).joinpath('01_DATA_transformed.parquet')) 

In [7]:
predictor_features = setup.predictor_features + ['mhier_NAICS_alt']
print(predictor_features)
with open(Path(setup.temp_path).joinpath('68_DATA_features_predict.pkl'), 'wb') as fout:
    pickle.dump(predictor_features, fout)

['NoEmp', 'CreateJob', 'LowDoc', 'DisbursementGross', 'new_business', 'urban_flag', 'franchise_flag', 'mhier_NAICS_alt']


In [8]:
best_params_df = pd.read_csv(Path(setup.temp_path).joinpath('03_REPORT_fit_parameter_selected.csv'))
best_params = best_params_df.to_dict(orient='records')[0]
best_params

{'subsample': 0.8,
 'scale_pos_weight': 1.0,
 'reg_lambda': 0.01,
 'reg_alpha': 0.001,
 'min_child_weight': 50,
 'max_depth': 8,
 'learning_rate': 0.1,
 'gamma': 2}

In [9]:
xgb_params = dict(best_params, 
                   **{'objective':"binary:logistic", 'n_estimators':setup.xgb_n_estimators})

In [10]:
embed_df = pd.read_parquet(Path(setup.temp_path).joinpath('63_DATA_embeddings_tsne_naics.parquet'))

##### NAICS levels to select NAICS-like clusters

In [11]:
naics_grp_stats = pd.read_csv(Path(setup.temp_path).joinpath('02_REPORT_naics_grp_stats_all.csv'))
naics_grp_k = naics_grp_stats['count_grp'].drop_duplicates().sort_values().to_list()
naics_grp_k

[20.0, 106.0, 354.0, 834.0]

In [12]:
cluster_cols = sorted([c for c in embed_df if c.startswith('cluster_')],
                      reverse=True)
cluster_cols

['cluster_834',
 'cluster_354',
 'cluster_106',
 'cluster_020',
 'cluster_010',
 'cluster_003']

In [13]:
sba_loans = sba_loans.drop(columns = cluster_cols, errors='ignore') \
    .merge(embed_df[['NAICS_orig'] + cluster_cols] \
               .rename(columns={'NAICS_orig':'NAICS'}),
           on='NAICS', how='left')

## Function to get metrics

In [14]:
def get_metrics(data, enc_features = ['NAICS', 'NAICS_sector']):
    
    # Alternative encoding
    train_df = data[data['dset'] == 'train']
    hier_enc = HierarchicalEncoder()
    hier_enc.fit(train_df[enc_features],train_df['target'])
    hier_col = hier_enc.transform(data[enc_features]) \
        .rename('mhier_NAICS_alt')
    
    # Append to data
    data = pd.concat([data.drop(columns='mhier_NAICS_alt', errors='ignore'), 
                       hier_col], axis=1)
    train_df = data[data['dset'] == 'train']
    test_df = data[data['dset'] == 'test']
    val_df = data[data['dset'] == 'val']
    
    # Model fit
    xgb_model = xgb.XGBClassifier(objective="binary:logistic",
                             n_estimators =  setup.xgb_n_estimators)
    xgb_model = xgb_model.set_params(**xgb_params)
    xgb_model.fit(train_df[predictor_features], train_df['target'],
              eval_set = [(train_df[predictor_features], train_df['target']),
                         (val_df[predictor_features], val_df['target'])])
    
    # Predict
    pred_df = pd.concat([data[['LoanNr_ChkDgt', 'dset', 'target', 'dset_naics_holdout']].reset_index(),
                     pd.DataFrame({'predict_prob':
                                   xgb_model.predict_proba(data[predictor_features])[:,1]})],
                    axis=1) \
    .set_index('index')
    
    # Decision threshold
    pred_train_df = pred_df[pred_df['dset'] == 'train']
    thresh_tune_data = sg_plot.get_f1_frame(pred_train_df['target'], 
                                        pred_train_df['predict_prob'])
    thresh_head = thresh_tune_data.sort_values('f1', ascending=False).head(2)
    best_thresh = thresh_head['thresh'].iloc[0]
    pred_df['predict_bin'] = np.where(pred_df['predict_prob'] >= best_thresh, 1, 0)
    
    # Metrics
    metrics_dset_df = pred_df.groupby('dset') \
        .apply(lambda x: sg_plot.dset_metrics(x.target, x.predict_bin, x.predict_prob)) \
        .reset_index()
    metrics_test_df = pred_df[pred_df['dset'] == 'test'] \
        .groupby(['dset', 'dset_naics_holdout']) \
        .apply(lambda x: sg_plot.dset_metrics(x.target, x.predict_bin, x.predict_prob)) \
        .reset_index()
    metrics_df = pd.concat([metrics_dset_df, metrics_test_df])
    
    return pred_df, metrics_df

## Loop to get predictions, metrics

In [15]:
# Hierarchical encode from different pairs
naics_sets = [['NAICS', cluster_cols[i]] \
              for i in range(len(cluster_cols))]
naics_sets

[['NAICS', 'cluster_834'],
 ['NAICS', 'cluster_354'],
 ['NAICS', 'cluster_106'],
 ['NAICS', 'cluster_020'],
 ['NAICS', 'cluster_010'],
 ['NAICS', 'cluster_003']]

In [16]:
%%capture
pred_df = pd.DataFrame()
metrics_df = pd.DataFrame()
for this_set in naics_sets:
    this_pred, this_met = get_metrics(sba_loans, this_set)
    this_pred['start_naics'] = this_set[1]
    pred_df = pd.concat([pred_df, this_pred])
    this_met['start_naics'] = this_set[1]
    metrics_df = pd.concat([metrics_df, this_met])

In [17]:
metrics_df[~metrics_df['dset_naics_holdout'].isna()]

,dset,accuracy_score,f1_score,precision_score,recall_score,average_precision_score,roc_auc_score,dset_naics_holdout,start_naics
0,test,0.660778,0.449924,0.337551,0.674452,0.384275,0.728603,0.0,cluster_834
1,test,0.667604,0.427372,0.311064,0.682599,0.339113,0.728745,1.0,cluster_834
0,test,0.660928,0.450052,0.337683,0.674504,0.383809,0.728442,0.0,cluster_354
1,test,0.704144,0.406464,0.319825,0.557482,0.335373,0.724007,1.0,cluster_354
0,test,0.659890,0.449868,0.337086,0.676065,0.383410,0.728403,0.0,cluster_106
1,test,0.700025,0.415223,0.321500,0.586076,0.346751,0.729413,1.0,cluster_106
0,test,0.660874,0.449631,0.337469,0.673464,0.383938,0.728759,0.0,cluster_020
1,test,0.692154,0.424739,0.321561,0.625412,0.356929,0.736225,1.0,cluster_020
0,test,0.661516,0.449660,0.337803,0.672268,0.383998,0.728885,0.0,cluster_010
1,test,0.699425,0.422107,0.324384,0.604094,0.356854,0.735481,1.0,cluster_010


In [18]:
pred_df.to_parquet(Path(setup.temp_path).joinpath('68_DATA_combined_predictions.parquet'))
metrics_df.to_csv(Path(setup.temp_path).joinpath('68_REPORT_metrics.csv'), index=True)